In [1]:
#importing data

%pylab inline
import neurosynth as ns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
#only once
#ns.dataset.download(path='.', unpack=True)

Populating the interactive namespace from numpy and matplotlib


In [2]:
#long runtime 
#from neurosynth.base.dataset import Dataset
dataset = ns.base.dataset.Dataset('database.txt');

In [3]:
#medium runtime
dataset.add_features('features.txt');

In [24]:
ids1 = dataset.get_studies(expression='alzheimer', frequency_threshold=0.001)

In [28]:
#data is saved in studies and 
df= pd.read_table('database.txt')
#t= mni_space['MNI'];
mni_space= df[df['space'].str.contains("MNI")];
x_vals = list(mni_space['x']);
y_vals = list(mni_space['y']);
z_vals = list(mni_space['z']);
study_ids = list(mni_space['id']);
unique_ids = list(set(study_ids)); #removing duplicates

term_table= pd.read_table('features.txt');
new_term_table = term_table.loc[term_table['pmid'].isin(unique_ids)];
terms = term_table.columns.values;

id_x_features = np.array(term_table['pmid']);

database_labels = df.columns.values;
print database_labels


['id' 'doi' 'x' 'y' 'z' 'space' 'peak_id' 'table_id' 'table_num' 'title'
 'authors' 'year' 'journal']


In [179]:
def initStudyDict():
    '''Creates a dictionary with Keys = study ids and Vals = tuples of x,y,z locations in NMI space'''
    
    '''Add parameter to smear xyz locations'''
    ids = {};
    c = 0;
    for i in study_ids:
        #print i
        if i not in ids:
            ids[i] = [(x_vals[c],y_vals[c],z_vals[c])];
            c +=1;
        elif (x_vals[c],y_vals[c],z_vals[c]) in ids[i]:
            c+=1
        else:
            ids[i].append((x_vals[c],y_vals[c],z_vals[c]))
    return ids     
id_mat = initStudyDict();

In [190]:
def isTerm(term):
    '''Checks if this term is in the neurosynth database'''
    if term in new_term_table:
        return True
    else:
        return False

def isLocation(coords):
    '''Checks if coordinate set (x,y,z) is mentioned in any studies'''
    if coords[0] in x_vals:
        ind = x_vals.index(coords[0]);
        if y_vals[ind] == coords[1] and z_vals[ind] == coords[2]:
            return True
        else:
            return False
    else: 
        return False
    
def isID(ID):
    '''Checks if ID is an ID with NMI coordinates'''
    if ID in id_mat:
        return True
    else:
        return False
    
    
def CoordtoIDs(coord):
    '''Uses the study dictionary above to find study ids from x,y,z coordinates'''
    if isLocation(coord):
        IDs = [];
        for i, coords in id_mat.items():
            if coord in coords:
                IDs.append(i)
        return IDs
    else:
        return "These coordinates don't match any studies"

def IDtoTerms(ID):
    '''Finds all of the term heat values of a given ID'''
    if isID(ID):
        ind= int(np.squeeze(np.where(id_x_features == ID)))
        #print ind
        return list(new_term_table.iloc[ind][1:])
    else:
        return 'Not an ID of a study in NMI space';

def IDtoCoords(ID):
    '''Finds coordinates associated with a given study ID'''
    if isID(ID):
        return id_mat[ID]
    else:
        return 'Not an ID of a study in NMI space'

#c = isID(15949653);
#print c

def CoordtoTerms(coord):
    '''Returns the vector of term heats for a given (x,y,z) coordinate set. 
    If there are multiple studies that mention the same coordinates, the average is taken.'''
    if isLocation(coord):
        ids = CoordtoIDs(coord)
        if len(ids) == 1:
            return IDtoTerms(ids[0])
        else: 
            temp = np.zeros((len(ids),3406));
            for i in xrange(len(ids)):
                temp[i,:] = IDtoTerms(ids[i]);
            return list(mean(temp,0))
    else:
        return []

    
def TermtoIDs(term,thresh):
    '''Matches a term to the IDs of studies that use that term above a given threshold'''
    if isTerm(term):
        term_all_ids = np.array(new_term_table[term]);
        id_inds = squeeze(np.where(term_all_ids>thresh));
        return id_x_features[id_inds]
    else:
        return []
    
def TermtoCoords(term,thresh):
    '''Finds the coordinates that are associated with a given term up to a given threshold'''
    ids = TermtoIDs(term,thresh);
    coords = [];
    coords = [IDtoCoords(i) for i in ids]
    coords= coords[0] #removing weird extra set of []
    return coords


#print TermtoIDs('schizophrenia',0.5)
c = TermtoCoords('schizophrenia',0.5)
print c

[(-6.0, 21.0, 41.0), (-6.0, 34.0, 41.0), (-6.0, 34.0, 28.0), (-6.0, 21.0, 28.0), (38.0, 15.0, -18.0), (36.0, 15.0, -18.0), (36.0, 26.0, -18.0), (36.0, 26.0, -16.0), (36.0, 17.0, -16.0), (36.0, 17.0, -13.0), (-44.0, 17.0, 23.0), (-44.0, 30.0, 23.0), (-44.0, 30.0, 21.0), (-42.0, 30.0, 21.0), (-42.0, 19.0, 21.0), (-42.0, 19.0, 27.0), (-4.0, 22.0, 50.0), (-4.0, 20.0, 50.0)]


In [10]:
#ns.network.coactivation(dataset, [[0, 20, 28]], threshold=0.1, output_dir='seed_test', prefix='acc_seed')

True


-34.0